In [ ]:
from keras.layers import Input, Dense
from keras.optimizers import SGD
from keras.models import Model

import numpy as np
import plotly.graph_objs as go
from plotly import offline as off
off.init_notebook_mode(connected=True)
import cufflinks as cf
import pandas as pd

cf.set_config_file(offline=True, world_readable=True)

# Quick Introduction to Neural Networks

## Artificial Neuron

<img src="http://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png" >

$$o_j^{(n)} = \varphi\left(\sum_{i\; in\; layer (n-1)}w_{ij}o_i^{(n-1)} \right)$$

## Step activation function
<img src="https://c.mql5.com/2/4/act1.png" align="middle">

## Logistic activation function

$$\varphi(x) = \frac{1}{1 - e^{-(x-b)}}$$

<img width= 300 src="http://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/SigmoidFunction.png/400px-SigmoidFunction.png" align="middle">

### Question: How to program an artificial neuron to calculate the *and* function?
<br>
<table>
<tr>
<th>$X$</th>
<th>$Y$</th>
<th>$X$ and $Y$</th>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td>0</td>
<td>1</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td style="text-align:center">1</td>
</tr>
</table>

## AND Neural Network
<img src="and1.png" align="middle">
# Define the toy dataset

In [ ]:
data = np.array([[1, 1], [1, 0], [0, 1], [0, 0]], dtype=np.float32)
labels = np.array([1, 0, 0, 0], dtype=np.float32)

# Define the model

In [ ]:
x = Input(shape=(2,))
y = Dense(1, activation='sigmoid')(x)

model = Model(inputs=x, outputs=y)

In [ ]:
def plotBoundary(model):
    x_list = np.arange(0.0, 1.0, 0.025)
    x1, x2 = np.meshgrid(x_list, x_list)
    x_vals = np.vstack([x1.ravel(), x2.ravel()]).T
    z = model.predict(x_vals)
    z = z.reshape(x1.shape).astype(np.float32)
    content = [go.Contour(x=x_list, y=x_list, z=z, showscale=False)]
    layout = go.Layout(title='Decision Boundary',
                       xaxis=dict(title='x1'),
                       yaxis=dict(title='x2')
                      )

    fig = go.Figure(data=content, layout=layout)
    off.iplot(fig)

In [ ]:
model.set_weights([[[10], [10]], [0]])
print(pd.DataFrame(np.hstack((data, model.predict(data))), columns=['x1', 'x2', 'prediction']))
plotBoundary(model)

## AND Neural Network
<img src="and2.png" align="middle">

### Question: How to program an artificial neuron to calculate the *xor* function?
<br/>
<table>
<tr>
<th>$X$</th>
<th>$Y$</th>
<th>$X$ xor $Y$</th>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td>0</td>
<td>1</td>
<td style="text-align:center">1</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
<td style="text-align:center">1</td>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td style="text-align:center">0</td>
</tr>
</table>

In [ ]:
data = np.array([[1, 1], [1, 0], [0, 1], [0, 0]], dtype=np.float32)
labels = np.array([0, 1, 1, 0], dtype=np.float32)

In [ ]:
model.set_weights([[[10], [10]], [-10]])
plotBoundary(model)

## Answer: It is impossible with only one neuron!. We need to use more than one neuron....
## Multilayer Neural Network
<img src="http://www.cs.nott.ac.uk/~gxk/courses/g5aiai/006neuralnetworks/images/ffnet.jpg">

In [ ]:
x = Input(shape=(2,))
h = Dense(2, activation='sigmoid')(x)
y = Dense(1, activation='sigmoid')(h)

model = Model(inputs=x, outputs=y)

model.compile(optimizer=SGD(lr=1),
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(data, labels, epochs=2000, verbose=0)

In [ ]:
plotBoundary(model)

## The Little Red Riding Hood Neural Network

<img src="http://themaleharem.com/wp-content/uploads/2014/06/Walter-crane-little-red-riding-hood-meets-the-wolf-in-the-woods.jpg">

In [ ]:
import pandas as pd

features = ['Big Ears', 'Big Teeth', 'Handsome', 'Wrinkled']
classes = ['Scream', 'Hug', 'Food', 'Kiss']
data = pd.DataFrame([[1,1,0,0,1,0,0,0],
                        [0,1,1,0,0,0,1,1],
                        [0,0,0,1,0,1,1,0]], columns=features + classes)

data

In [ ]:
x = Input(shape=(4,))
h = Dense(3, activation='sigmoid')(x)
y = Dense(4, activation='sigmoid')(h)

model = Model(inputs=x, outputs=y)

model.compile(optimizer=SGD(lr=1),
              loss='mean_squared_error',
              metrics=['accuracy'])
logs = model.fit(data[features].values, data[classes].values, epochs=1000, verbose=0)

In [ ]:
pd.DataFrame(logs.history).iplot()

In [ ]:
toeval = np.array([1, 1, 0, 0]).reshape(1, 4)
y_hat = model.predict(toeval)
print(pd.Series(toeval.ravel(), index=features))
print(pd.Series(y_hat.ravel(), index=classes))